In [ ]:
### plot coordinates in 3d slicer: 1.3.6.1.4.1.14519.5.2.1.6279.6001.183843376225716802567192412456
### perform coordinates conversion to voxels: https://notebook.community/flaviostutz/datascience-snippets/kaggle-lung-cancer-approach2/.ipynb_checkpoints/LungCancerDetection-checkpoint

In [ ]:
import glob
import json
import os
import numpy as np
import matplotlib.pyplot as plt
from utils import load_scan, normalize_planes
from PIL import Image
from skimage import measure
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from tqdm import tqdm

In [ ]:
MASK_FOLDER = r'C:/Users/ejrza/Downloads/mestrado/projeto/code/masters-object/POCS/preprocessing-tutorial/water-mask/*-mask.nrrd'
exams = glob.glob(MASK_FOLDER)
print(len(exams))
exams.sort()

In [ ]:
ANN_PATH = r'D:/masters_datasets/LUNA16-ann/mhd_original/dataset*'
anns = glob.glob(ANN_PATH)
print(len(anns))
anns.sort()

In [ ]:
processed = []
not_processed = []
fit = []
not_fit = []
pixels_out = 0
total_pixels = 0
summary = []

for ann in anns[0:1]: # entire dataset with cross val 10 fold (11)
    f = open(ann)
    category = json.load(f)
    for cat in category: # training, validation
        for im in tqdm(category[cat]): # every item - box, image, label (all label are 0)
            # verify if the file was already pre-processed
            current_file = im['image'].split('.mhd')[0] + '-mask.nrrd'
            if os.path.exists('water-mask/' + current_file):
                processed.append(current_file)

                ### perform nodule validation
                lung_mask, _, _ = load_scan('water-mask/' + current_file, meta=True)
                original, origin, spacing = load_scan(
                    'D:/masters_datasets/LUNA16-original/LUNA16_Images/'+im['image'].split('.mhd')[0]+'.mhd',
                    meta=True
                )

                ### draw the nodule mask
                axes = [lung_mask.shape[2], lung_mask.shape[1], lung_mask.shape[0]]
                nodule_mask = np.zeros(axes)

                p = lung_mask.transpose(2, 1, 0)

                print(np.unique(p, return_counts=True))

                print(im['image'])
                print(im['box'])

                for nodule in im['box']:
                    x = np.absolute(nodule[0] - origin[0])/spacing[0]
                    x0 = x - nodule[3]/2
                    x1 = x + nodule[3]/2
                    y = np.absolute(nodule[1] - origin[1])/spacing[1]
                    y0 = y - nodule[4]/2
                    y1 = y + nodule[4]/2
                    z = np.absolute(nodule[2] - origin[2])/spacing[2]
                    z0 = z - nodule[5]/2
                    z1 = z + nodule[5]/2

                    ### saving slices to analize
                    for z in range(int(z0), int(z1)):
                        image = normalize_planes(original[z])                        
                        image = Image.fromarray(image*255).convert('L')
                        image = image.crop((x0, y0, x1, y1))
                        image.save('output/'+ str(z) + 'slice.png')

                    for i in range(int(x0), int(x1)):
                        for j in range(int(y0), int(y1)):
                            for k in range(int(z0), int(z1)):

                                ### verify if all coordinates fit the lung mask
                                # print(lung_mask[k][j][i])
                                
                                # if lung_mask[k][j][i] != 1:
                                if p[i][j][k] != 1:
                                    pixels_out += 1
                                nodule_mask[i][j][k] = 1
                                total_pixels += 1

                    break

                summary.append({
                    "exam": im['image'],
                    "pixels_out": pixels_out,
                    "total_pixels": total_pixels
                })
                pixels_out = 0
                total_pixels = 0

                # p = lung_mask.transpose(2, 1, 0)
                verts, faces, normals, values = measure.marching_cubes(p, 0)

                fig = plt.figure(figsize=(10, 10))
                ax = fig.add_subplot(111, projection='3d')

                mesh = Poly3DCollection(verts[faces], alpha=0.20)
                face_color = [0.45, 0.45, 0.75]
                mesh.set_facecolor(face_color)
                ax.add_collection3d(mesh)

                ax.set_xlim(0, p.shape[0])
                ax.set_ylim(0, p.shape[1])
                ax.set_zlim(0, p.shape[2])

                ### nodule
                verts, faces, normals, values = measure.marching_cubes(nodule_mask, 0)
                mesh = Poly3DCollection(verts[faces], alpha=0.80)
                face_color = [1, 0, 0]
                mesh.set_facecolor(face_color)
                ax.add_collection3d(mesh)

                # ax.voxels(nodule_mask, facecolors=[1, 0, 0, 0.8])

                # ax.view_init(20, 90)

                break
            else:
                not_processed.append(current_file)
        break
    break
    
print(len(processed))
print(len(not_processed))

In [ ]:
print(summary)

In [ ]:
int('aa')

In [ ]:
import pandas as pd

result = pd.DataFrame(summary)
result.head()

In [ ]:
result['prop'] = result['pixels_out'] * 100/result['total_pixels']
result.head()

In [ ]:
hist = result['prop'].hist(bins=30)

In [ ]:
print(result.shape)
output = result.groupby(['prop'])['prop'].count()
sortedDF = output.sort_values(ascending=False)
sortedDF.head()

In [ ]:
result.shape

In [ ]:
result.to_csv('output/pixels_out.csv', index=False)